
**Vivacomet** Notebook simulates the environment and growth of organisms within it. The environment is represented as a grid, where each bin contains specific amounts of various molecules that organisms in that bin can utilize. The simulation takes into account the diffusion and advection of molecules, as well as the biomass of species. Additionally, Vivacomet simulates the growth of organisms in each bin, which is influenced by the availability of molecules in that bin.

In [9]:
#imports
import numpy as np
from processes.diffusion_field import DiffusionField
from processes.spatial_dfba import SpatialDFBA
from plots.field import plot_objective_flux, plot_fields_temporal
from vivarium.core.engine import Engine

In [10]:
#config

comets_config = {
    'total_time': 10,
    'bounds': [5, 5],
    'nbins': [2, 2],
}

# Parameters shared by both processes
shared_params = {
    'bounds': comets_config['bounds'],
    'nbins': comets_config['nbins'],
    'molecules': ['glucose', 'oxygen'],
}

species_info = [
    {
        "name": "Alteromonas",
        "model": '../data/Alteromonas_Model.xml',
        "flux_id_map": {
            "glucose": "EX_cpd00027_e0",
            "oxygen": "EX_cpd00007_e0"
        },
        "kinetic_params": {
            "glucose": (0.5, 0.0005),
            "oxygen": (0.3, 0.0005),
        },
        "fixed_bounds": {
            'EX_cpd00149_e0': (-10, 10)
        }
    },
    {
        "name": "ecoli",
        "model": '../data/iECW_1372.xml',
        "flux_id_map": {
            "glucose": "EX_glc__D_e",
            "oxygen": "EX_o2_e"
        },
        "kinetic_params": {
            "glucose": (0.4, 0.6),
            "oxygen": (0.25, 0.6),
        },
        "fixed_bounds": {
            'EX_fe3dhbzs_e': (0, 10)
        }
    }
]

# Specific parameters for Diffusion Field
diffusion_field_params = {
    **shared_params,
    'default_diffusion_dt': 0.001,
    'default_diffusion_rate': 2E-5,
    'diffusion': {
        'glucose': 6.7E-1,
        'oxygen': 2.0E-2,
    },
    'advection': {
        'glucose': (0.01, 0.02),
        'oxygen': (0.01, 0.01),
    }
}

# Specific parameters for Spatial DFBA
spatial_dfba_params = {
    **shared_params,
    'species_info': species_info
}

initial_state_config = {
    'random': {
        'glucose': 200.0,  # Max random value for glucose
        'oxygen': 200.0,   # Max random value for oxygen
        'species': {
            'ecoli': 0.5,   # Max random value for E. coli biomass
            'Alteromonas': 0.5   # Max random value for Alteromonas biomass
        }
    }
}


In [11]:
# create the two processes
diffusion_field = DiffusionField(parameters=diffusion_field_params)
spatial_dfba = SpatialDFBA(parameters=spatial_dfba_params)

# set the initial state for diffusion field
initial_state_diffusion_field = diffusion_field.initial_state({'random': 1.0})

# set the initial state for spatial dfba
initial_state_spatial_dfba = spatial_dfba.initial_state(initial_state_config)

# Merge the initial states
initial_state = {
    'fields': initial_state_diffusion_field['fields'],
    'species': initial_state_spatial_dfba['species'],
}

# make the composite simulation and run it
sim = Engine(
    processes={
        'diffusion_process': diffusion_field,
        'fba_process': spatial_dfba
    },
    topology={
        'diffusion_process': {
            'fields': ('fields',),
            'species': ('species',),
            'dimensions': ('dimensions',),
        },
        'fba_process': {
            'fields': ('fields',),
            'species': ('species',),
            'exchange_fluxes': ('exchange_fluxes',),
            'dimensions': ('dimensions',),
        }
    },
    initial_state=initial_state
)
sim.update(comets_config['total_time'])

# retrieve the results and plot them
data = sim.emitter.get_timeseries()
desired_time_points = [1, 3, comets_config['total_time'] - 1]

ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.


Loaded model for Alteromonas
Loaded model for ecoli

Simulation ID: 101ba564-2a73-11ef-9591-ba21c95c07d9
Created: 06/14/2024 at 13:25:15


/Users/amin/Desktop/VivaComet/myenv/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/Users/amin/Desktop/VivaComet/myenv/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/Users/amin/Desktop/VivaComet/myenv/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/Users/amin/Desktop/VivaComet/myenv/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/Users/amin/Desktop/VivaComet/myenv/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/Users/amin/Desktop/VivaComet/myenv/lib/python3.11/site-pack

Completed in 5.68 seconds


In [12]:
import numpy as np
import matplotlib.pyplot as plt
import imageio
import os
import io
import shutil
from IPython.display import display, HTML

# # Define configuration and parameters
# data = ...  # Your data here
# total_time = 200
# plot_fields = ["glucose", "oxygen", "ecoli", "Alteromonas"]
# molecule_colormaps = {"glucose": "Blues", "oxygen": "Greens", "ecoli": "Purples", "Alteromonas": "Oranges"}
# temp_dir = 'temp_images_fields'
# output_filename = 'comets_fields_over_time.gif'

def plot_fields_temporal_to_gif(data, total_time, plot_fields, molecule_colormaps, temp_dir):
    available_time_points = data["time"]
    valid_time_points = list(range(total_time + 1))

    num_fields = len(plot_fields)
    num_times = len(valid_time_points)
    images = []

    # Calculate global min and max for each field
    global_min = [np.inf] * num_fields
    global_max = [-np.inf] * num_fields

    # Precompute global min/max for field concentrations
    for time in valid_time_points:
        time_index = data["time"].index(time)

        for j, field_id in enumerate(plot_fields):
            current_field = data["fields"][field_id][time_index]
            global_min[j] = min(global_min[j], np.min(current_field))
            global_max[j] = max(global_max[j], np.max(current_field))

    # Plotting each field concentration for each time
    for time in valid_time_points:
        time_index = data["time"].index(time)
        fig, axs = plt.subplots(1, num_fields, figsize=(num_fields * 5, 5), squeeze=False)

        for j, field_id in enumerate(plot_fields):
            current_field = data["fields"][field_id][time_index]
            im = axs[0, j].imshow(current_field, cmap=molecule_colormaps[field_id], vmin=global_min[j], vmax=global_max[j])
            axs[0, j].set_title(field_id)
            axs[0, j].set_xticks([])
            axs[0, j].set_yticks([])
            plt.colorbar(im, ax=axs[0, j], fraction=0.046, pad=0.04)

        # Add the current time as text
        plt.suptitle(f'Time: {time}', fontsize=16)

        plt.tight_layout()
        buf = io.BytesIO()
        plt.savefig(buf, format='png', dpi=120)
        buf.seek(0)
        images.append(imageio.imread(buf))
        buf.close()
        plt.close(fig)

    # Save images to temporary directory
    for i, img in enumerate(images):
        imageio.imwrite(os.path.join(temp_dir, f'field_{i}.png'), img)

def combine_field_gifs(output_filename, temp_dir, num_images):
    combined_images = []
    for i in range(num_images):
        field_img = imageio.imread(os.path.join(temp_dir, f'field_{i}.png'))
        combined_images.append(field_img)

    imageio.mimsave(output_filename, combined_images, duration=0.5, loop=0)

    # Optionally display the GIF in a Jupyter notebook
    with open(output_filename, 'rb') as file:
        data = file.read()
        data_url = 'data:image/gif;base64,' + base64.b64encode(data).decode()
    display(HTML(f'<img src="{data_url}" alt="Combined GIF" style="max-width:100%;"/>'))

# Create a temporary directory
os.makedirs(temp_dir, exist_ok=True)

# Generate images
plot_fields_temporal_to_gif(data, total_time, plot_fields, molecule_colormaps, temp_dir)

# Combine GIFs
combine_field_gifs(output_filename, temp_dir, total_time + 1)

# Clean up temporary directory
shutil.rmtree(temp_dir)


KeyError: 'ecoli'